## Init

### Jupyter configurations

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Imports

In [64]:
import tensorflow as tf
import keras
import keras.backend as K
import numpy as np
import pandas as pd
import preprocessing as pp
import sys, inspect, argparse, importlib, traceback, re 
from copy import deepcopy

### Function: Accuracy Metric

In [3]:
# percentage of samples that exactly match
def exact_match_accuracy(y_true, y_pred):
    argmax_true = tf.math.argmax(y_true, axis=-1)            # onehot to index               (batch, width, onehot:int) -> (batch, width:int)
    argmax_pred = tf.math.argmax(y_pred, axis=-1)            # onehot to index               (batch, width, onehot:int) -> (batch, width:int)
    match_char = tf.math.equal(argmax_true, argmax_pred)     # match characters              (batch, width:int) -> (batch, width:bool)
    match_word = tf.math.reduce_all(match_char, axis=-1)     # require all character in sample to match      (batch, width:bool) -> (batch:bool)
    match_int = tf.cast(match_word, tf.float32)              # bool to int                                   (batch:bool) -> (batch:float)
    return tf.reduce_mean(match_int)                         # percentage of samples that are an exact match (batch:float) -> float

### Function: Log function

In [4]:
verbose = False
def log(*l, **d): 
    if verbose: print(*l, **d)
        
training_history = []

def training_log(x, y, a, b, e, l, m):
    training_history.append({'x':x, 'y':y, 'architecture':a, 'batch size':b, 'epochs':e, 'loss':l, 'accuracy':m})

## Preprocess Data

### Tokens and Characters

In [5]:
# tokens used to communicate non character entities
# tokens = ['<Padding>', '<Go>', '<EndOfString>', '<UnknownChar>', '<SurveyNum>', '<SurveyName>', '<LineName>', '<SurveyType>', '<PrimaryDataType>', '<SecondaryDataType>', '<TertiaryDataType>', '<Quaternary>', '<File_Range>', '<First_SP_CDP>', '<Last_SP_CDP>', '<CompletionYear>', '<TenureType>', '<Operator Name>', '<GSQBarcode>', '<EnergySource>', '<LookupDOSFilePath>', '<Source Of Data>']
tokens = ['<Padding>', '<Go>', '<EndOfString>', '<UnknownChar>']

# get set of characters to be used, use static preset list of characters
#available_chars = list(" ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890-_().,\\/\"':&")
available_chars = list(" ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890-_().,\\/\"':&")

# generate character to int and int to character maps
char_to_int = {c: i for i, c in enumerate(tokens + available_chars)}
int_to_char = {i: c for c, i in char_to_int.items()}
char_count = len(char_to_int) # number of character available

### Read raw data

In [6]:
raw_source_file = 'SHUP 2D Files Training Data.csv'

# read raw training data
data_df = pd.read_csv(raw_source_file, dtype=str)
data = {feature:data_df[feature].values for feature in data_df.columns.values}

# split strings into words
delimiters = r'( |_|-|\.|\,|/|\\|\(|\)|&|:|\'|")' # any of '-_().,\\/\"':& '
replacement = r'\0\g<1>\0' # surround delimiter with splitting token

# split strings acording to RegEx
data['LookupDOSFilePath_Words'] = np.array([re.sub(delimiters, replacement, s).split() for s in data['LookupDOSFilePath']])
data['FileName_Words'] = np.array([re.sub(delimiters, replacement, s).split() for s in data['FileName']])
data['LineName_Words'] = np.array([re.sub(delimiters, replacement, s).split() for s in data['LineName']])

print(data.keys())

dict_keys(['Unique Record ID', 'FileName', 'Original_FileName', 'SurveyNum', 'SurveyName', 'LineName', 'SurveyType', 'PrimaryDataType', 'SecondaryDataType', 'TertiaryDataType', 'Quaternary', 'File_Range', 'First_SP_CDP', 'Last_SP_CDP', 'CompletionYear', 'TenureType', 'Operator Name', 'GSQBarcode', 'EnergySource', 'LookupDOSFilePath', 'Source Of Data', 'LookupDOSFilePath_Words', 'FileName_Words', 'LineName_Words'])


### Function: Vectorize Strings

In [7]:
def vectorize_data(data):
    if type(data) == str:
        return [char_to_int[char] for char in data.upper()]
    
    else:
        try:
            return [vectorize_data(d) for d in data]
        except Exception as e:
            #traceback.print_exc()
            #print(data, type(data))
            return []


def devectorise_data(data):
        
    length = data.shape[0]
    
    data = data.reshape(length, -1)
    strings = np.full((length,), '', dtype=object)
    
    for i in range(length):
        strings[i] = ''.join([int_to_char[int(i)] for i in data[i]])
    
    return strings
    
    
#     ndim = data.ndim
#     if data.dtype != object:
#         data = data.astype(object)
    
#     # decode vector into string
#     if ndim == 1:
#         return ''.join([int_to_char[int(i)] for i in data])
    
#     # go to next level
#     else:
#         for i in range(len(data)):
#             data[i] = devectorise_data(data[i])
            
#         return data

### Function: Add Padding Tokens

In [8]:
# find size of largest array across each dimension to computer shape of bounding ndarray
def size(data):
    
    if type(data) == int:
        return ()
    
    this_size = len(data)
    
    if this_size > 0:
        inner_sizes = np.array([size(d) for d in data])
        inner_sizes = tuple(np.amax(inner_sizes, axis=0))
    else:
        inner_sizes = ()
    
    return (this_size,) + inner_sizes
    
    
def insert_vector(matrix, data, indices):
    if type(data) == int:
        matrix[indices] = data
    else:
        for i in range(len(data)):
            insert_vector(matrix, data[i], indices + (i,))
    

def pad_vector_data(data, pad_token, pad_shape=None):
    
    shape = size(data)
    if pad_shape != None:
        shape = tuple(np.maximum(pad_shape, shape))

    # empty matrix
    matrix = np.full(shape, pad_token, np.int32)

    insert_vector(matrix, data, ())

    return matrix

### Function: Split dataset

In [9]:
def split(data, sizes):
    sizes = list(sizes)
    
    for i in range(1, len(sizes)):
        sizes[i] += sizes[i-1]
    
    slices = [slice(i,j) for i, j in zip([0]+sizes, sizes)]
    
    return [data[s] for s in slices]

### Function: Shuffle dataset

In [10]:
def shuffle(*data):
    order = np.arange(len(data[0]))         # default order of elements
    np.random.shuffle(order)                # randomise order
    return [d[order] for d in data]         # new array with items in the randimised order

### Function: Extract relevant data

In [11]:
def extract(*keys, **cuts):
    
    onehots, shapes = [], []
        
    for key in keys:
        
        # get data from dictionary
        onehot = onehot_data[key]

        # apply cuts
        cut = cuts.get(key, [[None]])
        cut = [slice(*c) for c in cut]
        cut = len(onehot.shape)*[slice(None)] + cut + [slice(None)]
        cut = tuple(cut[-len(onehot.shape):])
        onehot = onehot[cut]
        
        # calculate shape
        shape = (None, *onehot.shape[1:])[-3:]

        onehots.append(onehot)
        shapes.append(shape)

    return onehots, shapes
    
def split_and_shuffle(*onehots, sizes=None, shuffle_before=False, shuffle_after=True):
    sizes = sizes or [None]
    key_count = len(onehots)
    subset_count = len(sizes)
    
    if shuffle_before:
        onehots = shuffle(*onehots)

    onehots_subsets = np.full((key_count, subset_count), None)
    onehots_subsets[:,:] = [split(onehot, sizes) for onehot in onehots]

    if shuffle_after:
        for i in range(subset_count):
            onehots_subsets[:,i] = shuffle(*onehots_subsets[:,i])

    return onehots_subsets

### Perform preprocessing

In [12]:
# extract LookupDOSFilePath for speccial processing

vectorized_data = {f: vectorize_data(data[f])    for f in data}
padded_data =     {f: pad_vector_data(vectorized_data[f], char_to_int['<Padding>'])    for f in vectorized_data}
onehot_data =     {f: keras.utils.to_categorical(padded_data[f], char_count)    for f in padded_data}

for f in onehot_data: print(f"'{f}':".ljust(30), onehot_data[f].shape)

'Unique Record ID':            (23903, 6, 53)
'FileName':                    (23903, 87, 53)
'Original_FileName':           (23903, 71, 53)
'SurveyNum':                   (23903, 5, 53)
'SurveyName':                  (23903, 39, 53)
'LineName':                    (23903, 23, 53)
'SurveyType':                  (23903, 6, 53)
'PrimaryDataType':             (23903, 14, 53)
'SecondaryDataType':           (23903, 36, 53)
'TertiaryDataType':            (23903, 17, 53)
'Quaternary':                  (23903, 8, 53)
'File_Range':                  (23903, 13, 53)
'First_SP_CDP':                (23903, 8, 53)
'Last_SP_CDP':                 (23903, 7, 53)
'CompletionYear':              (23903, 4, 53)
'TenureType':                  (23903, 3, 53)
'Operator Name':               (23903, 47, 53)
'GSQBarcode':                  (23903, 17, 53)
'EnergySource':                (23903, 29, 53)
'LookupDOSFilePath':           (23903, 181, 53)
'Source Of Data':              (23903, 8, 53)
'LookupDOSFilePath_Wo

## Function: Test and show samlpe output

In [13]:
def test(model, x_test=None, y_test=None, x_preview=None, y_preview=None):
    
    if (x_preview is not None) and (y_preview is not None):
        t_size = len(x_preview)
        
        p_one_hot = model.predict(x_preview)
        p_vector = np.argmax(p_one_hot, -1)
        p_vector = p_vector.reshape((t_size, -1))
        p_strings = devectorise_data(p_vector)

        y_vector = np.argmax(y_preview, -1)
        y_vector = y_vector.reshape((t_size, -1))
        y_strings = devectorise_data(y_vector)

        x_vector = np.argmax(x_preview, -1)
        x_vector = x_vector.reshape((t_size, -1))
        x_strings = devectorise_data(x_vector)

        n_strings = [f'{i}. ' for i in range(t_size)]
        x_strings = [re.sub('(<Padding>)+', ' ', s).strip() for s in x_strings]
        y_strings = [re.sub('(<Padding>)+', ' ', s).strip() for s in y_strings]
        p_strings = [re.sub('(<Padding>)+', ' ', s).strip() for s in p_strings]
        n_w, x_w, y_w, p_w = max([len(s) for s in n_strings]), max([len(s) for s in x_strings]), max([len(s) for s in y_strings]), max([len(s) for s in p_strings])
        y_p_strings = ["'  '".join([n.ljust(n_w), x.ljust(x_w), y.ljust(y_w), p.ljust(p_w), str(y==p)]) for n, x, y, p in zip(n_strings, x_strings, y_strings, p_strings)]

        print(*y_p_strings, sep='\n', end='\n\n')

    if (x_test is not None) and (y_test is not None):
        
        # metric names
        metrics = [model.loss] + model.metrics
        
        # accuracy on entire training set
        accuracies = model.evaluate(x_test, y_test)
        print(*list(zip(metrics, accuracies)), sep='\n', end='\n\n') # evaluate and list loss and each metric

        return accuracies[0], accuracies[-1]

## Training Parameters

In [157]:
models = {}
embeddings = {}

In [180]:
architecture = ''

metrics = ['mean_absolute_error', 'categorical_accuracy', exact_match_accuracy] # binary_accuracy
loss = 'categorical_crossentropy' # poisson mean_squared_logarithmic_error categorical_crossentropy

embed_loss='categorical_crossentropy'
embed_metrics=['accuracy', 'mean_absolute_error', 'categorical_accuracy', exact_match_accuracy]

## Character Embedding

### Auto Encoder Character Data

In [158]:
# create offset input and output sequences to training a preditive embedding model.

(x_char_onehot,), ((embed_word_count, embed_char_count, embed_ones_count),) = extract('LookupDOSFilePath')
(x_char_onehot,) = shuffle(x_char_onehot)
shape = x_char_onehot.shape
x_embed_size = len(x_char_onehot)

# create columns of padding tokens
padding = np.full((*shape[:-2], 1), char_to_int['<Padding>'])
#padding = np.array([[char_to_int['<Padding>']]] * x_embed_size)
padding = keras.utils.to_categorical(padding, char_count)
padding = padding.reshape(*shape[:-2], 1, shape[-1])

# 'abcd' -> ('_abcd', 'abcd_')
x_embed_train = np.concatenate((x_char_onehot, padding), axis=-2)
y_embed_train = np.concatenate((padding, x_char_onehot), axis=-2)
x_embed_test = np.concatenate((x_char_onehot, padding), axis=-2)
y_embed_test = np.concatenate((padding, x_char_onehot), axis=-2)

embed_char_count += 1

### Auto Encoder: Input, Hidden, Output

In [159]:
architecture = 'Character-Embedding'

model_E_D_NN = keras.Sequential()
model_E_D_NN.add(keras.layers.Dense(character_embedding_size, name='lh', input_shape=(embed_char_count, char_count,)))
model_E_D_NN.add(keras.layers.Dense(char_count, activation='sigmoid', name='lo'))
#model_E_D_NN.add(keras.layers.Dropout(0.001))
model_E_D_NN.compile(optimizer='adam', loss=embed_loss, metrics=embed_metrics)
models[architecture] = model_E_D_NN
print(model_E_D_NN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lh (Dense)                   (None, 182, 10)           540       
_________________________________________________________________
lo (Dense)                   (None, 182, 53)           583       
Total params: 1,123
Trainable params: 1,123
Non-trainable params: 0
_________________________________________________________________
None


### Train Encoder and Decoder

In [160]:
# x -> y   predictive
epochs = 2
batch_size = 16
models['Character-Embedding'].fit(x_embed_train, y_embed_train, batch_size=batch_size, epochs=epochs)
test(models['Character-Embedding'], None, None, x_embed_train[:1], y_embed_train[:1])

Epoch 1/2
23903/23903 [==============================] - 20s 822us/step - loss: 2.1194 - acc: 0.4504 - mean_absolute_error: 0.1151 - categorical_accuracy: 0.4504 - exact_match_accuracy: 0.0000e+00
Epoch 2/2
23903/23903 [==============================] - 17s 704us/step - loss: 1.4233 - acc: 0.5622 - mean_absolute_error: 0.0324 - categorical_accuracy: 0.5622 - exact_match_accuracy: 0.0000e+00
0. '  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1980\BRANSBY\SEGY\BRANSBY_80-KHW_RAW_MIGRATED_QR011264_167289.SGY'  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1980\BRANSBY\SEGY\BRANSBY_80-KHW_RAW_MIGRATED_QR011264_167289.SGY'  'A_SSUA\ED_SORSG_AUOROS__SEDTAED_SUUROADETATA\198A\OTA_\GA_SEGA\OTA_\GD989CSADOTAD_FEOTASED_O8\\\10D\11\911_EG'  'False



In [162]:
# x -> x   direct
epochs = 5
batch_size = 16
models['Character-Embedding'].fit(x_embed_train, x_embed_train, batch_size=batch_size, epochs=epochs)
test(models['Character-Embedding'], None, None, x_embed_train[:1], x_embed_train[:1])

Epoch 1/5
23903/23903 [==============================] - 18s 746us/step - loss: 0.2023 - acc: 0.9332 - mean_absolute_error: 0.0205 - categorical_accuracy: 0.9332 - exact_match_accuracy: 0.0303
Epoch 2/5
23903/23903 [==============================] - 17s 697us/step - loss: 0.0148 - acc: 0.9891 - mean_absolute_error: 0.0174 - categorical_accuracy: 0.9891 - exact_match_accuracy: 0.1965
Epoch 3/5
23903/23903 [==============================] - 17s 696us/step - loss: 0.0093 - acc: 0.9892 - mean_absolute_error: 0.0171 - categorical_accuracy: 0.9892 - exact_match_accuracy: 0.1987
Epoch 4/5
23903/23903 [==============================] - 17s 723us/step - loss: 0.0046 - acc: 0.9973 - mean_absolute_error: 0.0173 - categorical_accuracy: 0.9973 - exact_match_accuracy: 0.7994
Epoch 5/5
23903/23903 [==============================] - 17s 725us/step - loss: 2.4996e-04 - acc: 1.0000 - mean_absolute_error: 0.0173 - categorical_accuracy: 1.0000 - exact_match_accuracy: 1.0000
0. '  '\SHUP\2D_SURVEYS\PROCESS

In [163]:
# w1 = model.layers[0].get_weights()
# w2 = model.layers[1].get_weights()

# w = [np.copy(w1[0]), np.zeros(w1[1].shape)]
# wi = [np.linalg.pinv(w1[0]), np.zeros(w2[1].shape)]

# m = keras.Sequential()
# m.add(keras.layers.Dense(character_embedding_size, activation='linear', name='lh', input_shape=(embed_char_count, voc_size,)))
# m.add(keras.layers.Dense(voc_size, activation='sigmoid', name='lo'))
# m.compile(optimizer='adam', loss=embed_loss, metrics=embed_metrics)

# m.layers[0].set_weights(w)
# m.layers[1].set_weights(wi)

# encode_weights, decode_weights = w, wi

# accuracy = model.evaluate(embed_test_x, embed_test_x)
# metric_names = [embed_loss] + embed_metrics
# dict(zip(metric_names, accuracy))

embeddings['Character-Embedding'] = [models['Character-Embedding'].layers[0].get_weights(), models['Character-Embedding'].layers[1].get_weights()]

In [164]:
# test
test(models['Character-Embedding'], x_embed_train, x_embed_train, x_embed_train[:5], x_embed_train[:5])

0. '  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1980\BRANSBY\SEGY\BRANSBY_80-KHW_RAW_MIGRATED_QR011264_167289.SGY                                         '  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1980\BRANSBY\SEGY\BRANSBY_80-KHW_RAW_MIGRATED_QR011264_167289.SGY                                         '  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1980\BRANSBY\SEGY\BRANSBY_80-KHW_RAW_MIGRATED_QR011264_167289.SGY                                         '  'True
1. '  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1990\GREGORY_2D_JACKSON_AND_BOLAN_3D\SEGY\GREGORY_2D_JACKSON_AND_BOLAN_3D_93-EEH_FINAL_QR020041_170077.SGY'  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1990\GREGORY_2D_JACKSON_AND_BOLAN_3D\SEGY\GREGORY_2D_JACKSON_AND_BOLAN_3D_93-EEH_FINAL_QR020041_170077.SGY'  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1990\GREGORY_2D_JACKSON_AND_BOLAN_3D\SEGY\GREGORY_2D_JACKSON_AND_BOLAN_3D_93-EEH_FINAL_QR020041_170077.SGY'  'True
2. '  '\SHUP\2D_SURVEYS\PROCESSED_AND_SUPPORT_DATA\1

(0.00015475044764043055, 1.0)

## Word Embedding

### Auto Encoder Folder Data

In [134]:
class WordEmbedding():
    def __init__(self, x_name, compression=0.1):

        # extract and shuffle data
        (x_word_onehot,), ((embed_word_count, embed_char_count, embed_ones_count),) = extract(x_name)
        (x_word_onehot,) = shuffle(x_word_onehot)

        # if data is not split into words, create extra word diemnstion
        if(embed_word_count == None):
            embed_word_count = 1
            x_word_onehot = np.expand_dims(x_word_onehot, 1)
        
        self.x_word_onehot = x_word_onehot

        # define embedding sizes
        self.character_embedding_size = character_embedding_size
        self.word_embedding_size = int(embed_char_count * self.character_embedding_size * compression) + 1
        c_size, w_size = self.character_embedding_size, self.word_embedding_size
        
        
        # define model

        # character embedding
        self.l_encode_character = keras.layers.Dense(c_size, name='char_encode', input_shape=(embed_word_count, embed_char_count, embed_ones_count))
        self.l_char_to_word = keras.layers.Reshape((embed_word_count, embed_char_count * c_size,))

        # word auto encoder
        self.l_encode_word = keras.layers.Dense(w_size, name='lh', input_shape=(embed_char_count, char_count,))
        self.l_decode_word = keras.layers.Dense(embed_char_count * c_size, activation='sigmoid', name='lo')

        # character de embedding
        self.l_word_to_char = keras.layers.Reshape((embed_word_count, embed_char_count, c_size))
        self.l_decode_character = keras.layers.Dense(embed_ones_count, activation='sigmoid', name='char_decode')

        self.model = keras.Sequential([
            self.l_encode_character,
            self.l_char_to_word,
            self.l_encode_word,
            self.l_decode_word,
            self.l_word_to_char,
            self.l_decode_character,
        ])

        self.model.compile(optimizer='adam', loss=embed_loss, metrics=embed_metrics)

        
        # set pre-trained character embedding
        self.l_encode_character.set_weights(deepcopy(embeddings['Character-Embedding'][0]))
        self.l_decode_character.set_weights(deepcopy(embeddings['Character-Embedding'][1]))

        print(self.model.summary())
    
    
    def train(self, epochs=5, batch_size=32):
        # perform training
        self.model.fit(self.x_word_onehot, self.x_word_onehot, batch_size=batch_size, epochs=epochs)
        test(self.model, self.x_word_onehot, self.x_word_onehot, self.x_word_onehot[:3], self.x_word_onehot[:3])

    def apply_encode(self, new_encode_character, new_encode_word):
        new_encode_character.set_weights(deepcopy(self.l_encode_character.get_weights()))
        new_encode_word.set_weights(deepcopy(self.l_encode_word.get_weights()))

    def apply_decode(self, new_decode_word, new_decode_character):
        new_decode_word.set_weights(deepcopy(self.l_decode_word.get_weights()))
        new_decode_character.set_weights(deepcopy(self.l_decode_character.get_weights()))
        

### Train word embedding

#### LineName

In [165]:
embeddings['LineName_Embedding'] = WordEmbedding('LineName', 0.15)
embeddings['LineName_Embedding'].train(10, 16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_encode (Dense)          (None, 1, 23, 10)         540       
_________________________________________________________________
reshape_93 (Reshape)         (None, 1, 230)            0         
_________________________________________________________________
lh (Dense)                   (None, 1, 35)             8085      
_________________________________________________________________
lo (Dense)                   (None, 1, 230)            8280      
_________________________________________________________________
reshape_94 (Reshape)         (None, 1, 23, 10)         0         
_________________________________________________________________
char_decode (Dense)          (None, 1, 23, 53)         583       
Total params: 17,488
Trainable params: 17,488
Non-trainable params: 0
_________________________________________________________________
None
E

#### FileName_Words

In [168]:
embeddings['FileName_Words_Embedding'] = WordEmbedding('FileName_Words', 0.15)
embeddings['FileName_Words_Embedding'].train(8, 16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_encode (Dense)          (None, 29, 13, 10)        540       
_________________________________________________________________
reshape_99 (Reshape)         (None, 29, 130)           0         
_________________________________________________________________
lh (Dense)                   (None, 29, 20)            2620      
_________________________________________________________________
lo (Dense)                   (None, 29, 130)           2730      
_________________________________________________________________
reshape_100 (Reshape)        (None, 29, 13, 10)        0         
_________________________________________________________________
char_decode (Dense)          (None, 29, 13, 53)        583       
Total params: 6,473
Trainable params: 6,473
Non-trainable params: 0
_________________________________________________________________
None
Epo

#### SurveyName

In [167]:
embeddings['SurveyName_Embedding'] = WordEmbedding('SurveyName', 0.15)
embeddings['SurveyName_Embedding'].train(10, 16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_encode (Dense)          (None, 1, 39, 10)         540       
_________________________________________________________________
reshape_97 (Reshape)         (None, 1, 390)            0         
_________________________________________________________________
lh (Dense)                   (None, 1, 59)             23069     
_________________________________________________________________
lo (Dense)                   (None, 1, 390)            23400     
_________________________________________________________________
reshape_98 (Reshape)         (None, 1, 39, 10)         0         
_________________________________________________________________
char_decode (Dense)          (None, 1, 39, 53)         583       
Total params: 47,592
Trainable params: 47,592
Non-trainable params: 0
_________________________________________________________________
None
E

In [169]:
embeddings['SurveyName_Embedding'].train(2, 16)

Epoch 1/2
23903/23903 [==============================] - 24s 1ms/step - loss: 0.0052 - acc: 0.9991 - mean_absolute_error: 0.0158 - categorical_accuracy: 0.9991 - exact_match_accuracy: 0.9693: 5s - loss: 0.0052 - acc: 0.9991 - mean_absolute_error: 0 - ETA: 1s - loss: 0.0052 - acc: 0.9991 - mean_absolute_error: 0.0158 - categorical_accuracy: 0.9991 - exact_m
Epoch 2/2
23903/23903 [==============================] - 23s 953us/step - loss: 0.0046 - acc: 0.9992 - mean_absolute_error: 0.0160 - categorical_accuracy: 0.9992 - exact_match_accuracy: 0.9730
0. '  'R80       '  'R80       '  'R80       '  'True
1. '  'PARENJELLY'  'PARENJELLY'  'PARENJELLY'  'True
2. '  'AR91      '  'AR91      '  'AR91      '  'True

23903/23903 [==============================] - 5s 190us/step
('categorical_crossentropy', 0.004410482474722919)
('accuracy', 0.999087123777489)
('mean_absolute_error', 0.016016207693145223)
('categorical_accuracy', 0.999087123777489)
(<function exact_match_accuracy at 0x00000211DAA0DE

## Models

### P-NN: Input, Embedding, Output

In [1]:
architecture = 'P-NN'

model_P_NN = keras.Sequential()
model_P_NN.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
model_P_NN.add(keras.layers.Flatten())                                                                       # flatten to 1D per sample
model_P_NN.add(keras.layers.Dense(y_shape_char*y_shape_ones, activation='exponential', name='lo'))           # dense layer
model_P_NN.add(keras.layers.Dropout(0.001))                                                                  # dropout to prevent overfitting
model_P_NN.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_P_NN.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_P_NN
print(model_P_NN.summary())

NameError: name 'keras' is not defined

### FF-NN: Input, Embedding, Hidden, Output

In [37]:
architecture = 'FF-NN'
hidden_size = (y_shape_ones*embedding_size + y_shape_char*y_shape_ones) // 2

model_FF_NN = keras.Sequential()
model_FF_NN.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
model_FF_NN.add(keras.layers.Flatten())                                                                       # flatten to 1D per sample
model_FF_NN.add(keras.layers.Dense(hidden_size, activation='exponential', name='lh'))                         # dense layer
model_FF_NN.add(keras.layers.Dropout(0.2))                                                                    # dropout to prevent overfitting
model_FF_NN.add(keras.layers.Dense(y_shape_char*y_shape_ones, activation='exponential', name='lo'))           # dense layer
model_FF_NN.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_FF_NN.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_FF_NN
print(model_FF_NN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
le (Embedding)               (None, 23, 20)            1580      
_________________________________________________________________
flatten_5 (Flatten)          (None, 460)               0         
_________________________________________________________________
lh (Dense)                   (None, 1264)              582704    
_________________________________________________________________
dropout_6 (Dropout)          (None, 1264)              0         
_________________________________________________________________
lo (Dense)                   (None, 948)               1199220   
_________________________________________________________________
reshape_7 (Reshape)          (None, 12, 79)            0         
Total params: 1,783,504
Trainable params: 1,783,504
Non-trainable params: 0
_________________________________________________________________


### LSTM-RNN1: Input, Embedding, (LSTM), Output

In [83]:
architecture = 'LSTM-RNN1'
lstm_hidden_size = embedding_size * 15

model_LSTM_RNN1 = keras.Sequential()
model_LSTM_RNN1.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
#model_LSTM_RNN1.add(keras.layers.Dropout(0.2))                                                                    # dropout to prevent overfitting
model_LSTM_RNN1.add(keras.activation.exponential())
model_LSTM_RNN1.add(keras.layers.LSTM(y_shape_char * y_shape_ones, activation='exponential', implementation=2, unroll=True))                # lstm recurrent cell
model_LSTM_RNN1.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_LSTM_RNN1.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_LSTM_RNN1
print(model_LSTM_RNN1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
le (Embedding)               (None, 23, 20)            1580      
_________________________________________________________________
lstm_7 (LSTM)                (None, 948)               3674448   
_________________________________________________________________
reshape_15 (Reshape)         (None, 12, 79)            0         
Total params: 3,676,028
Trainable params: 3,676,028
Non-trainable params: 0
_________________________________________________________________
None


### LSTM-RNN2: Input, Embedding, (LSTM), Output

In [261]:
architecture = 'LSTM-RNN2'

x_name, y_name = 'LineName', 'LineName'
cuts = {x_name:[[None, 6]] , y_name:[[None, 6]]}
(x_onehot, y_onehot), ((x_word_count, x_char_count, x_ones_count),(y_word_count, y_char_count, y_ones_count)) = extract(x_name, y_name, **cuts)

lstm_hidden_size = y_char_count * character_embedding_size * 2

model_LSTM_RNN2 = keras.Sequential()
model_LSTM_RNN2.add(keras.layers.Dense(character_embedding_size, name='char_encode', trainable=False, input_shape=(x_char_count, x_ones_count)))                 # embed characters into dense embedded space
#model_LSTM_RNN2.add(keras.layers.Dropout(0.2))                                                                            # dropout to prevent overfitting
model_LSTM_RNN2.add(keras.layers.LSTM(lstm_hidden_size, activation='sigmoid', implementation=2, unroll=True, name='lstm1'))     # lstm recurrent cell
#model_LSTM_RNN2.add(keras.layers.Dropout(0.2))                                                                            # dropout to prevent overfitting
model_LSTM_RNN2.add(keras.layers.Dense(y_char_count * character_embedding_size, activation='sigmoid', name='decode'))  # dense layer, decode/de-embed
model_LSTM_RNN2.add(keras.layers.Reshape((y_char_count, character_embedding_size)))                                        # un flatten
model_LSTM_RNN2.add(keras.layers.Dense(y_ones_count, activation='sigmoid', trainable=False, name='char_decode'))                             # dense layer, decode/de-embed
model_LSTM_RNN2.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_LSTM_RNN2
print(model_LSTM_RNN2.summary())

# set pretrained embedding weights
model_LSTM_RNN2.layers[0].set_weights(char_encode_weights)
model_LSTM_RNN2.layers[-1].set_weights(char_decode_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_encode (Dense)          (None, 6, 20)             1080      
_________________________________________________________________
lstm1 (LSTM)                 (None, 240)               250560    
_________________________________________________________________
decode (Dense)               (None, 120)               28920     
_________________________________________________________________
reshape_21 (Reshape)         (None, 6, 20)             0         
_________________________________________________________________
char_decode (Dense)          (None, 6, 53)             1113      
Total params: 281,673
Trainable params: 279,480
Non-trainable params: 2,193
_________________________________________________________________
None


### LSTM-RNN3: Input, Embed Characters, Embed Words, (LSTM), De-embed Words, De-embed Characters, Output

In [181]:
architecture = 'LSTM-RNN3'

x_name, y_name = 'FileName_Words', 'LineName'
cuts = {} # {x_name:[[None, 6]] , y_name:[[None, 6]]}
(x_onehot, y_onehot), ((x_word_count, x_char_count, x_ones_count),(y_word_count, y_char_count, y_ones_count)) = extract(x_name, y_name, **cuts)

x_embed = embeddings[x_name+'_Embedding']
y_embed = embeddings[y_name+'_Embedding']

lstm_hidden_size = 100


# character embedding
l_encode_character = keras.layers.Dense(   x_embed.character_embedding_size, input_shape=(x_word_count, x_char_count, x_ones_count))

# word embedding
l_char_to_word =     keras.layers.Reshape( (x_word_count, x_char_count * x_embed.character_embedding_size,))
l_encode_word =      keras.layers.Dense(   x_embed.word_embedding_size)

# lstm processing
l_lstm =             keras.layers.LSTM(    lstm_hidden_size,   activation='sigmoid',   implementation=2,   unroll=True)
l_decode =           keras.layers.Dense(   y_embed.word_embedding_size,   activation='sigmoid')

l_decode_word =      keras.layers.Dense(   y_char_count * y_embed.character_embedding_size,   activation='sigmoid')
l_word_to_char =     keras.layers.Reshape( (y_char_count, character_embedding_size))

l_decode_character = keras.layers.Dense(   y_ones_count,   activation='sigmoid')


models[architecture] = keras.Sequential([
    l_encode_character,
    l_char_to_word,
    l_encode_word,
    l_lstm,
    l_decode,
    l_decode_word,
    l_word_to_char,
    l_decode_character,
])

models[architecture].compile(optimizer='adam', loss=loss, metrics=metrics)
print(models[architecture].summary())

# set pretrained embedding weights
x_embed.apply_encode(l_encode_character, l_encode_word)
y_embed.apply_decode(l_decode_word, l_decode_character)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 29, 13, 10)        540       
_________________________________________________________________
reshape_107 (Reshape)        (None, 29, 130)           0         
_________________________________________________________________
dense_48 (Dense)             (None, 29, 20)            2620      
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               48400     
_________________________________________________________________
dense_49 (Dense)             (None, 35)                3535      
_________________________________________________________________
dense_50 (Dense)             (None, 230)               8280      
_________________________________________________________________
reshape_108 (Reshape)        (None, 23, 10)            0         
__________

### GRU-RNN1: Input, Embedding, (GRU), Output

In [32]:
architecture = 'GRU-RNN1'
lstm_hidden_size = voc_size * 15

model_GRU_RNN1 = keras.Sequential()
model_GRU_RNN1.add(keras.layers.Embedding(y_shape_ones, embedding_size, name='le', input_length=x_shape_char))   # embed characters into dense embedded space
#model_GRU_RNN1.add(keras.layers.Dropout(0.2))                                                                    # dropout to prevent overfitting
model_GRU_RNN1.add(keras.layers.GRU(y_shape_char * y_shape_ones, activation='relu', implementation=2, unroll=True))                # lstm recurrent cell
model_GRU_RNN1.add(keras.layers.Reshape((y_shape_char, y_shape_ones)))                                           # un flatten
model_GRU_RNN1.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_GRU_RNN1
print(model_GRU_RNN1.summary())

NameError: name 'voc_size' is not defined

### GRU-RNN2: Imput Embedding, (GRU), Decoder, Output

In [188]:
architecture = 'GRU-RNN2'

x_name, y_name = 'LineName', 'LineName'
cuts = {x_name:[[None, 12]] , y_name:[[None, 12]]}
(x_onehot, y_onehot), ((x_word_count, x_char_count, x_ones_count),(y_word_count, y_char_count, y_ones_count)) = extract(x_name, y_name, **cuts)

lstm_hidden_size = y_char_count * character_embedding_size + 40

model_GRU_RNN2 = keras.Sequential()
model_GRU_RNN2.add(keras.layers.Dense(character_embedding_size, name='char_encode', trainable=False, input_shape=(x_char_count, x_ones_count)))                 # embed characters into dense embedded space
model_GRU_RNN2.add(keras.layers.Dropout(0.2))                                                                            # dropout to prevent overfitting
model_GRU_RNN2.add(keras.layers.GRU(lstm_hidden_size, activation='sigmoid', implementation=2, unroll=True, name='gru1'))     # gru recurrent cell
model_GRU_RNN2.add(keras.layers.Dropout(0.2))                                                                            # dropout to prevent overfitting
model_GRU_RNN2.add(keras.layers.Dense(y_char_count * character_embedding_size, activation='sigmoid', name='decode'))  # dense layer, decode/de-embed
model_GRU_RNN2.add(keras.layers.Reshape((y_char_count, character_embedding_size)))                                        # un flatten
model_GRU_RNN2.add(keras.layers.Dense(y_ones_count, activation='sigmoid', trainable=False, name='char_decode'))                             # dense layer, decode/de-embed
model_GRU_RNN2.compile(optimizer='adam', loss=loss, metrics=metrics)
models[architecture] = model_GRU_RNN2
print(model_GRU_RNN2.summary())

# set pretrained embedding weights
model_GRU_RNN2.layers[0].set_weights(char_encode_weights)
model_GRU_RNN2.layers[-1].set_weights(char_decode_weights)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_encode (Dense)          (None, 12, 10)            540       
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 10)            0         
_________________________________________________________________
gru1 (GRU)                   (None, 160)               82080     
_________________________________________________________________
dropout_2 (Dropout)          (None, 160)               0         
_________________________________________________________________
decode (Dense)               (None, 120)               19320     
_________________________________________________________________
reshape_4 (Reshape)          (None, 12, 10)            0         
___________________________

### GRU

In [ ]:
architecture = 'GRU-RNN3'

x_name, y_name = 'FileName_Words', 'SurveyName'
cuts = {} # {x_name:[[None, 6]] , y_name:[[None, 6]]}
(x_onehot, y_onehot), ((x_word_count, x_char_count, x_ones_count),(y_word_count, y_char_count, y_ones_count)) = extract(x_name, y_name, **cuts)

x_embed = embeddings[x_name+'_Embedding']
y_embed = embeddings[y_name+'_Embedding']

gru_hidden_size = int(y_embed.word_embedding_size * 3)


# character embedding
l_encode_character = keras.layers.Dense(   x_embed.character_embedding_size, input_shape=(x_word_count, x_char_count, x_ones_count))

# word embedding
l_char_to_word =     keras.layers.Reshape( (x_word_count, x_char_count * x_embed.character_embedding_size,))
l_encode_word =      keras.layers.Dense(   x_embed.word_embedding_size)

# lstm processing
l_lstm =             keras.layers.GRU(     gru_hidden_size,   activation='sigmoid',   implementation=2,   unroll=True)
l_decode =           keras.layers.Dense(   y_embed.word_embedding_size,   activation='sigmoid')

l_decode_word =      keras.layers.Dense(   y_char_count * y_embed.character_embedding_size,   activation='sigmoid')
l_word_to_char =     keras.layers.Reshape( (y_char_count, character_embedding_size))

l_decode_character = keras.layers.Dense(   y_ones_count,   activation='sigmoid')


models[architecture] = keras.Sequential([
    l_encode_character,
    l_char_to_word,
    l_encode_word,
    l_lstm,
    l_decode,
    l_decode_word,
    l_word_to_char,
    l_decode_character,
])

models[architecture].compile(optimizer='adam', loss=loss, metrics=metrics)
print(models[architecture].summary())

# set pretrained embedding weights
x_embed.apply_encode(l_encode_character, l_encode_word)
y_embed.apply_decode(l_decode_word, l_decode_character)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_77 (Dense)             (None, 29, 13, 10)        540       
_________________________________________________________________
reshape_119 (Reshape)        (None, 29, 130)           0         
_________________________________________________________________
dense_78 (Dense)             (None, 29, 20)            2620      
_________________________________________________________________
gru_6 (GRU)                  (None, 177)               105138    
_________________________________________________________________
dense_79 (Dense)             (None, 59)                10502     
_________________________________________________________________
dense_80 (Dense)             (None, 390)               23400     
_________________________________________________________________
reshape_120 (Reshape)        (None, 39, 10)            0         
__________

## Save/Restore weights

In [184]:
#DE = models['E-D-NN'].get_weights()
#model_GRU_1 = model
#model_GRU_2 = model
#model_GRU_3 = model

In [133]:
#model.set_weights(GRU)
#model = model_GRU_3
#models['E-D-NN'].set_weights(DE)

## Run and Evaluate

In [185]:
def train(model, epochs, batch_size):
    
    # split and shuffle data
    ((x_train, x_test, x_preview), (y_train, y_test, y_preview)) = split_and_shuffle(x_onehot, y_onehot, sizes=(20000, 1000, 15))

    # train
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
    
    # test
    l, a = test(model, x_test, y_test, x_preview, y_preview)
    
    # show training history
    training_log(x_name, y_name, architecture, batch_size, epochs, l, a)
    print(*training_history[::-1], sep='\n')

In [183]:
train(models['GRU-RNN3'], 70, 64)

Epoch 1/70
20000/20000 [==============================] - 40s 2ms/step - loss: 1.1868 - mean_absolute_error: 0.0188 - categorical_accuracy: 0.7446 - exact_match_accuracy: 0.0000e+00
Epoch 2/70
20000/20000 [==============================] - 33s 2ms/step - loss: 0.7810 - mean_absolute_error: 0.0188 - categorical_accuracy: 0.8012 - exact_match_accuracy: 5.0000e-05
Epoch 3/70
20000/20000 [==============================] - 32s 2ms/step - loss: 0.6752 - mean_absolute_error: 0.0188 - categorical_accuracy: 0.8186 - exact_match_accuracy: 2.0000e-04
Epoch 4/70
20000/20000 [==============================] - 33s 2ms/step - loss: 0.5983 - mean_absolute_error: 0.0187 - categorical_accuracy: 0.8357 - exact_match_accuracy: 7.5000e-04
Epoch 5/70
20000/20000 [==============================] - 32s 2ms/step - loss: 0.5341 - mean_absolute_error: 0.0186 - categorical_accuracy: 0.8517 - exact_match_accuracy: 0.0021
Epoch 6/70
20000/20000 [==============================] - 32s 2ms/step - loss: 0.4829 - mean_a

20000/20000 [==============================] - 32s 2ms/step - loss: 0.0477 - mean_absolute_error: 0.0094 - categorical_accuracy: 0.9907 - exact_match_accuracy: 0.8559
Epoch 47/70
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0461 - mean_absolute_error: 0.0092 - categorical_accuracy: 0.9909 - exact_match_accuracy: 0.8601
Epoch 48/70
20000/20000 [==============================] - 35s 2ms/step - loss: 0.0450 - mean_absolute_error: 0.0091 - categorical_accuracy: 0.9912 - exact_match_accuracy: 0.8637
Epoch 49/70
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0428 - mean_absolute_error: 0.0090 - categorical_accuracy: 0.9917 - exact_match_accuracy: 0.8714
Epoch 50/70
20000/20000 [==============================] - 35s 2ms/step - loss: 0.0415 - mean_absolute_error: 0.0088 - categorical_accuracy: 0.9920 - exact_match_accuracy: 0.8749
Epoch 51/70
20000/20000 [==============================] - 35s 2ms/step - loss: 0.0404 - mean_absolute_error: 0.0087 

In [192]:
train(models['GRU-RNN3'], 70, 64)

Epoch 1/70
20000/20000 [==============================] - 62s 3ms/step - loss: 1.1959 - mean_absolute_error: 0.0188 - categorical_accuracy: 0.7444 - exact_match_accuracy: 0.0000e+00
Epoch 2/70
20000/20000 [==============================] - 47s 2ms/step - loss: 0.7983 - mean_absolute_error: 0.0188 - categorical_accuracy: 0.8000 - exact_match_accuracy: 0.0000e+00
Epoch 3/70
20000/20000 [==============================] - 51s 3ms/step - loss: 0.6771 - mean_absolute_error: 0.0188 - categorical_accuracy: 0.8180 - exact_match_accuracy: 1.0000e-04
Epoch 4/70
20000/20000 [==============================] - 50s 2ms/step - loss: 0.6052 - mean_absolute_error: 0.0187 - categorical_accuracy: 0.8344 - exact_match_accuracy: 6.5000e-04
Epoch 5/70
20000/20000 [==============================] - 48s 2ms/step - loss: 0.5431 - mean_absolute_error: 0.0186 - categorical_accuracy: 0.8507 - exact_match_accuracy: 0.0013
Epoch 6/70
20000/20000 [==============================] - 49s 2ms/step - loss: 0.4891 - mean_a

KeyboardInterrupt: 